In [260]:
# Problem 2a

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import statistics as st

data = pd.read_csv("kc_house_data.csv")

def scale(data):
    # standardize every feature in dataset
    feat_scaled = data.drop(columns=["id", "date", "price", "zipcode"])
    feat_names = feat_scaled.columns.tolist()
    for feat in feat_names:
        feat_scaled[feat] = (feat_scaled[feat] - st.mean(feat_scaled[feat])) / st.stdev(feat_scaled[feat])

    # scale price by dividing by 1000
    price_scaled = data["price"] / 1000
    
    return feat_scaled, price_scaled

X_scaled, y_scaled = scale(data)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=5)

# create model and fit to training data
MLR_model = LinearRegression()
MLR_model.fit(X_train, y_train)
y_train_pred = MLR_model.predict(X_train)

# calculate coefficients, MSE, and R2
coeffs = MLR_model.coef_
feat_names = X_scaled.columns.tolist()
coeffs_table = pd.DataFrame(coeffs, feat_names, columns=["Coefficients"])

mse = mean_squared_error(y_train, y_train_pred)
r2 = r2_score(y_train, y_train_pred)

print(coeffs_table)
print("\nMSE:",round(mse, 3))
print("R2:",round(r2, 3))

               Coefficients
bedrooms      -2.827722e+01
bathrooms      3.139104e+01
sqft_living    2.843715e+15
sqft_lot       4.534507e+00
floors         4.927442e-01
waterfront     5.070269e+01
view           3.888898e+01
condition      2.108529e+01
grade          1.139387e+02
sqft_above    -2.563970e+15
sqft_basement -1.370319e+15
yr_built      -7.289534e+01
yr_renovated   1.094533e+01
lat            7.784047e+01
long          -1.578261e+01
sqft_living15  1.753381e+01
sqft_lot15    -9.951207e+00

MSE: 40897.45
R2: 0.696


In [261]:
# Problem 2b

# use trained model for test set
y_test_pred = MLR_model.predict(X_test)

mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print("MSE:",round(mse, 3))
print("R2:",round(r2, 3))

MSE: 41747.783
R2: 0.694


### Problem 2c

sqft_living, sqft_above, and sqft_basement all contributed the most to the linear regression model by far because they had the greatest coefficients. As far as error, the model fit the data somewhat well, with an MSE of about 40879, which is decent considering the size of the data set. The R2 value of 0.696 shows that there is a moderately strong correlation between the features and price. The MSE and R2 for the testing are almost identical to the training, which shows that the fitted model is equally as accurate with the testing data.

In [262]:
# Problem 3a

import numpy as np

# convert train-test splits to 2d numpy arrays
X_trn = X_train.values
y_trn = y_train.values
X_tst = X_test.values
y_tst = y_test.values

# add column of ones as first column
def add_ones(x):
    ones_col = np.ones(len(x))
    x = np.c_[ones_col,x]
    return x

# calculate theta using closed-form solution
def closed_solution(X, y):

    XT = np.transpose(X)
    theta = np.linalg.inv(XT @ X) @ XT @ y
    
    return theta

# h theta of (x) function
def h_theta(X, theta):
    
    return X @ theta

# add bias column to Xs
X_trn = add_ones(X_trn)
X_tst = add_ones(X_tst)

# get theta from closed form solution
theta = closed_solution(X_trn, y_trn)

# get prediction using training data
y_trn_predict = h_theta(X_trn, theta)

# get prediction using testing data
y_tst_predict = h_theta(X_tst, theta)

In [263]:
# Problem 3b

train_mse = mean_squared_error(y_trn, y_trn_predict)
test_mse = mean_squared_error(y_tst, y_tst_predict)
train_r2 = r2_score(y_train, y_trn_predict)
test_r2 = r2_score(y_test, y_tst_predict)

print("MSE for training:",round(train_mse, 3))
print("R2 for training:",round(train_r2, 3))

print("\nMSE for testing:",round(test_mse, 3))
print("R2 for testing:",round(test_r2, 3))

MSE for training: 41585.425
R2 for training: 0.69

MSE for testing: 42578.37
R2 for testing: 0.688


The MSE and R2 for both the training and testing data is nearly identical to the corresponding MSE and R2 values from Problem 3, with an MSE of around 41,000 to 42,000 and an R2 of about 0.69. Implementing the closed-form solution worked effectively the same as the built-in linear regression package.

In [264]:
# Problem 4a

from sklearn.preprocessing import PolynomialFeatures

def poly_reg(X, Y, p):
    
    # make X 2D with 1 column
    X = X.reshape(-1, 1)
    
    # add extra features of x up to the degree of p
    poly = PolynomialFeatures(degree=p)
    X_poly = poly.fit_transform(X)
    
    theta = closed_solution(X_poly, Y)
    poly_pred = h_theta(X_poly, theta)

    return poly_pred

In [265]:
# Problem 4b

# get only the sqft_living feature for 
X_trn_sqft = X_train["sqft_living"].values
X_tst_sqft = X_test["sqft_living"].values

evals = np.empty((5,4))

for i in range(5):  
    # get polynomial predictions
    y_trn_pred = poly_reg(X_trn_sqft, y_trn, i+1)
    y_tst_pred = poly_reg(X_tst_sqft, y_tst, i+1)

    # get MSEs and R2s for training and testing predictions
    evals[i][0] = round(mean_squared_error(y_trn, y_trn_pred),3)
    evals[i][1] = round(r2_score(y_trn, y_trn_pred),3) 
    evals[i][2] = round(mean_squared_error(y_tst, y_tst_pred),3)
    evals[i][3] = round(r2_score(y_tst, y_tst_pred),3)

cols = ["Train MSE","Train R2","Test MSE","Test R2"]
rows = ["Deg. 1","Deg. 2","Deg. 3","Deg. 4","Deg. 5"]
report_table = pd.DataFrame(evals, columns=cols, index=rows)
print(report_table)

        Train MSE  Train R2   Test MSE  Test R2
Deg. 1  68389.622     0.491  68191.915    0.500
Deg. 2  62962.356     0.531  62963.083    0.539
Deg. 3  61922.848     0.539  62866.131    0.539
Deg. 4  61388.736     0.543  62554.140    0.541
Deg. 5  60781.205     0.548  61523.281    0.549


As the degree increases, MSE decreases consistently and R2 increases consistently. Overall the model is better fit at higher degree polynomials.

In [266]:
# Problem 5a

def GD(X, y, alpha, iters):
    
    # initialize theta
    theta = closed_solution(X, y)
    
    # gradient descent
    for i in range(1,iters):
        theta = theta - (alpha * (2 / len(X)) * np.transpose((X @ theta) - y) @ X)
    
    return theta

In [267]:
# Problem 5b

iters = [10,50,100]
alphas = [0.01,0.1,0.5]

trn_mse = np.empty((3,3))
trn_r2 = np.empty((3,3))
tst_mse = np.empty((3,3))
tst_r2 = np.empty((3,3))

# get predictions and MSE/R2 for all alphas and iterations
for i in range(len(iters)):
    for a in range(len(alphas)):
        trained_GD = GD(X_trn, y_trn, alphas[a], iters[i])
        
        pred_trn = h_theta(X_trn, trained_GD)
        pred_tst = h_theta(X_tst, trained_GD)
        
        trn_mse[i][a] = mean_squared_error(y_trn, pred_trn)
        tst_mse[i][a] = mean_squared_error(y_tst, pred_tst)
        trn_r2[i][a] = r2_score(y_trn, pred_trn)
        tst_r2[i][a] = r2_score(y_tst, pred_tst)

# make tables for training and testing MSE and R2
cols = ["a=0.01","a=0.1","a=0.5"]
rows = ["10 iter.","50 iter.","100 iter."]
trn_mse_table = pd.DataFrame(trn_mse, columns=cols, index=rows)
trn_r2_table = pd.DataFrame(trn_r2, columns=cols, index=rows)
tst_mse_table = pd.DataFrame(tst_mse, columns=cols, index=rows)
tst_r2_table = pd.DataFrame(trn_r2, columns=cols, index=rows)

# print tables
print("Training MSE:\n",trn_mse_table)
print("\nTraining R2:\n",trn_r2_table)
print("\nTesting MSE:\n",tst_mse_table)
print("\nTesting R2:\n",tst_r2_table)

Training MSE:
                  a=0.01         a=0.1          a=0.5
10 iter.   41268.599049  40921.796044   9.093472e+12
50 iter.   40970.293199  40896.557540   5.103008e+61
100 iter.  40920.203402  40896.329246  4.407547e+122

Training R2:
              a=0.01     a=0.1          a=0.5
10 iter.   0.692853  0.695434  -6.767943e+07
50 iter.   0.695073  0.695622  -3.797985e+56
100 iter.  0.695446  0.695623 -3.280378e+117

Testing MSE:
                  a=0.01         a=0.1          a=0.5
10 iter.   42226.593136  41764.982834   9.544558e+12
50 iter.   41835.295787  41722.043443   5.356127e+61
100 iter.  41764.093799  41718.414828  4.626169e+122

Testing R2:
              a=0.01     a=0.1          a=0.5
10 iter.   0.692853  0.695434  -6.767943e+07
50 iter.   0.695073  0.695622  -3.797985e+56
100 iter.  0.695446  0.695623 -3.280378e+117


### Problem 5c

As both the iterations and alpha increase, the MSE decreases and R2 increases. However, at alpha = 0.5, MSE is extremely high and R2 is completely out of bounds. This is because when the step size is too large, the gradient descent may overshoot, fail to converage, or diverage, resulting in an inaccurate model. Of the given iterations and alpha levels, 100 iterations and alpha = 0.1 gave the lowest MSE and highest R2. These likely lead the model very close to the global minimum.

In [268]:
# Problem 6b

def GD_ridge(X, y, alpha, iters, lmbda):
    # initialize theta
    theta = closed_solution(X, y)
    
    # gradient descent with ridge regression
    for j in range(1,iters):
        theta = theta * (1 - (alpha * lmbda)) - (alpha * (2/len(X)) * np.transpose((X @ theta) - y) @ X)
    
    return theta

# test that GD_ridge works with arbitrary lambda
gd_ridge = GD_ridge(X_trn, y_trn, 0.1, 100, 10)
print(gd_ridge)

[ 89.98067069 -44.43545388 -60.1687786  -57.72635572 -18.70596645
 -44.55297394   2.66171158  -7.90475559  23.26574182 -52.31737776
 -61.32728237  -5.04700107 -56.57741253   4.61302053  15.21090766
 -39.86059206 -54.34472173 -20.69022941]


In [269]:
# Problem 6c

# create X with random uniform distribution
X = np.empty((1000,1))
for i in range(1000):
    X[i] = np.random.uniform(-2, 2)
    
# generate gaussian distribution with 1000 entries with mean 1 and stdev 0.25
# points will fall between 0 and 2
e = np.random.normal(1, 0.25, 1000)
e = e.reshape(-1,1)

# get ones for first term in Y
ones = np.ones(1000)
ones = ones.reshape(-1,1)

Y = ones + (2*X) + e

# closed form solution for ridge regression
def closed_ridge(X, Y, lmbda):
    XT = np.transpose(X)
    theta = np.linalg.inv((XT @ X) + (lmbda * np.eye(2))) @ XT @ Y
    return theta

X = add_ones(X)

# get linear prediction and slope/MSE/R2
theta_linear = closed_solution(X, Y)
pred_linear = h_theta(X, theta_linear)
print("Linear Regression")
print("slope:",round(theta_linear[1][0], 3))
print("MSE:",round(mean_squared_error(Y, pred_linear), 3))
print("R2:",round(r2_score(Y, pred_linear), 3))

# get ridge prediction and slope/MSE/R2 for all levels of lambda
for l in [1,10,100,1000,10000]:
    theta_ridge = closed_ridge(X, Y, l)
    pred_ridge = h_theta(X, theta_ridge)
    print("\nRidge Regression for lambda =",l)
    print("slope:",round(theta_ridge[1][0], 3))
    print("MSE:",round(mean_squared_error(Y, pred_ridge), 3))
    print("R2:",round(r2_score(Y, pred_ridge), 3))

Linear Regression
slope: 2.002
MSE: 0.063
R2: 0.989

Ridge Regression for lambda = 1
slope: 2.0
MSE: 0.063
R2: 0.989

Ridge Regression for lambda = 10
slope: 1.987
MSE: 0.063
R2: 0.988

Ridge Regression for lambda = 100
slope: 1.865
MSE: 0.121
R2: 0.978

Ridge Regression for lambda = 1000
slope: 1.153
MSE: 2.026
R2: 0.629

Ridge Regression for lambda = 10000
slope: 0.239
MSE: 7.564
R2: -0.386


As the regularization parameter lambda increases, slope decreases, MSE increases, and R2 decreases, indicating an overall decrease in model fit. A lambda that is too high causes underfitting, because it reduces the model complexity too much.